## Collaborative Filtering

In [ ]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [ ]:
book_ratings = pd.read_csv('BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

* Explore both datasets

* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


* create the instance of the NearestNeighbors class

* fit the NearestNeighbors using'df_book_features'

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

* Apply the function to book of your choice